In [25]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from itertools import repeat, chain
import matplotlib.colors as colors

In [26]:
# Order of all possible bins for plotting
order = ['(0, 10]', '(10, 20]', '(20, 30]', '(30, 40]', '(40, 50]',
         '(50, 60]', '(60, 70]', '(70, 80]', '(80, 90]', '(90, 100]',
         '(100, 150]', '(150, 200]', '(200, 500]', '(500, 1000]']

In [44]:
def get_order(df, order, bbin):
    # Make sorted list of bins present
    ordered = []
    reach_bins = list(set(df[bbin].to_list()))

    for i in order:
        if i in reach_bins:
            ordered.append(i)
            
    return ordered

In [41]:
def read_output(data_path):
    df_c = []

    for file in os.listdir(data_path):
        if not file.startswith('.'):
            if 'cent' in file:
                df_c.append(file)
                
    # Make df dataframe
    d = []

    for file in df_c:
        data = pd.read_parquet(os.path.join(data_path, file))
        d.append(data)
        
    df = pd.concat(d, axis=0, ignore_index=True)
    return df

In [29]:
def grouped_weighted_avg(values, weights):
    return (values * weights).sum() / weights.sum()

In [ ]:
# def read_in_centiles

## Get data

### Max

In [38]:
## Setup
bwidth = 'min'
bbin = 'Bin_Min'
data_path = '../narrow_rivers_PIXC_output/PIXC_v2_0_HUC2_01_2025_03_02_' + bwidth

In [39]:
df = read_output(data_path)

In [45]:
ordered = get_order(df=df, order=order, bbin=bbin)

In [47]:
# expanded = {}
# data = {}

# for width in ordered[3:4]:
#     for thresh in df.threshold.unique():
#         coverage = np.array(df[(df[binn] == width) & (df['threshold'] == thresh)]['coverage'].to_list())
#         counts = np.array(df[(df[binn] == width) & (df['threshold'] == thresh)]['count'].to_list())

#         new = np.array(list(chain.from_iterable(repeat(value, count) for value, count in zip(coverage, counts))))

#         data[i] = new
        
#     expanded[binn] = data

In [ ]:
# zero_ten_thresh_30 = np.array(df[(df['Bin_Max'] == ordered[3]) & (df['threshold'] == 0.3)]['coverage'].to_list())

In [11]:
# zero_ten_thresh_30_count = np.array(df[(df['Bin_Max'] == ordered[3]) & (df['threshold'] == 0.3)]['count'].to_list())

In [12]:
# test = np.array(list(chain.from_iterable(repeat(value, count) for value, count in zip(zero_ten_thresh_30, zero_ten_thresh_30_count))))

In [50]:
df_mean = df.groupby([bbin,'quantile','threshold'])[list(df.columns)].apply(lambda x: grouped_weighted_avg(x['coverage'], x['count']))

In [51]:
df_mean = df_mean.reset_index().rename(columns={0: 'weighted_coverage'})

In [52]:
# Convert to percentage
df_mean[['weighted_coverage', 'threshold']] = df_mean[['weighted_coverage', 'threshold']]*100

In [53]:
# thresholds = [10, 30, 50, 70, 90]
df_subset = df_mean[(df_mean['threshold'] == 10) | (df_mean['threshold'] == 30) |
                    (df_mean['threshold'] == 50) | (df_mean['threshold'] == 70) |
                    (df_mean['threshold'] == 90)]

In [60]:
df_subset

,Bin_Min,quantile,threshold,weighted_coverage
0,"(0, 10]",0.00,10.0,0.000036
2,"(0, 10]",0.00,30.0,0.000021
4,"(0, 10]",0.00,50.0,0.000018
6,"(0, 10]",0.00,70.0,0.000000
8,"(0, 10]",0.00,90.0,0.000000
...,...,...,...,...
10791,"(90, 100]",0.99,10.0,89.408124
10793,"(90, 100]",0.99,30.0,88.850723
10795,"(90, 100]",0.99,50.0,87.892116
10797,"(90, 100]",0.99,70.0,85.644963


In [ ]:
# df_crop = df_subset[df_subset[binn] != '(200, 500]']

In [ ]:
# reach_thirty = df_mean[df_mean['threshold'] == 30]
# reach_thirty = pd.DataFrame(reach_thirty, columns=['NHDPlus HR df with > 10% detection in all nodes'])

In [ ]:
# reach_ten = df_mean[df_mean['threshold'] == 10]
# reach_ten = pd.DataFrame(reach_ten, columns=['NHDPlus HR df with > 10% detection in all nodes'])

In [ ]:
# ten_list = []
# for binn in ordered:
#     print(binn)
#     coverage = reach_ten[reach_ten['Bin'] == binn].coverage.to_list()
#     ten_list.append(coverage)

In [ ]:
# reach_fifty = df_mean[df_mean['threshold'] == 50]
# reach_fifty = pd.DataFrame(reach_fifty, columns=['NHDPlus HR df with > 10% detection in all nodes'])

## Plot data

In [ ]:
# # Make sorted list of bins present
# ordered = []
# node_bins = node_w_zero_mean.Bin.to_list()
# for i in order:
#     if i in node_bins:
#         ordered.append(i)
# # ordered

In [54]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [55]:
cmap = plt.get_cmap('cubehelix')
new_cmap = truncate_colormap(cmap, 0.2, 0.75)

In [ ]:
save_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/figures_for_2025_03_03/'

#### New

In [ ]:
# fig, ax = plt.subplots(figsize=(12,6))
# sns.boxplot(x=binn, y='weighted_coverage',
#             data=reach_thirty, order=ordered, color='#2D7038',
#             saturation=1, showmeans=True,
#             fliersize=2, linewidth=2,
#             boxprops=dict(alpha=0.8),
#             meanprops={'marker':'o',
#                        'markerfacecolor':'white',
#                        'markeredgecolor':'k',
#                        'markersize':'8'})

# plt.title('Bieger et al. width == ' + width + ', 30% threshold', fontsize=16,
#            fontweight='bold')
# plt.xticks(fontsize=14, fontweight='semibold', rotation=-45)
# plt.yticks(fontsize=14, fontweight='semibold',)

# plt.xlabel('Width Bin [m]', fontsize=16,
#            fontweight='bold', labelpad=20)
# plt.ylabel('Detected [%]', fontsize=16,
#            fontweight='bold')

# plt.savefig(fname=save_path + 'thirty_' + width + '.png', dpi=300, bbox_inches='tight')

In [64]:
ordered

['(0, 10]',
 '(10, 20]',
 '(20, 30]',
 '(30, 40]',
 '(40, 50]',
 '(50, 60]',
 '(60, 70]',
 '(70, 80]',
 '(80, 90]',
 '(90, 100]',
 '(100, 150]',
 '(150, 200]']

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
sns.boxplot(x=bbin, y='weighted_coverage', hue='threshold',
            data=df_subset, 
            order=ordered, palette=new_cmap,
            saturation=1, showmeans=True,
            fliersize=2, legend=False,
            boxprops=dict(alpha=0.8),
            meanprops={'marker':'o',
                       'markerfacecolor':'white',
                       'markeredgecolor':'k',
                       'markersize':'4'})

plt.title('Bieger et al. width: ' + bwidth + '\nOLD WAY', fontsize=16,
           fontweight='bold')

plt.xticks(fontsize=14, fontweight='semibold', rotation=-45)
plt.yticks(fontsize=14, fontweight='semibold',)

plt.xlabel('Width Bin [m]', fontsize=16,
           fontweight='bold', labelpad=20)
plt.ylabel('Detected [%]', fontsize=16,
           fontweight='bold')

# plt.legend(title='Threshold [%]', #loc='upper left',
#            title_fontproperties={'weight': 'bold',
#                                  'size': 14},
#            prop={'weight': 'semibold',
#                  'size': 12})

# plt.savefig(fname=save_path + 'all_' + width + '.png', dpi=300, bbox_inches='tight')

### For Later

In [ ]:
# Group by the 'Bin' column and compute the weighted mean for each group
def weighted_mean(group):
    return (group['count'] * group['mean']).sum() / group['count'].sum()

# Apply the weighted mean function to each group
weighted_means = nodes_desc.groupby('Bin').apply(weighted_mean).reset_index(name='Weighted Mean')

In [ ]:
weighted_means

In [ ]:
# Group by the 'Bin' column
def weighted_std(group, weighted_mean):
    # Compute the weighted variance
    weighted_variance = ((group['count'] * ((group['mean'] - weighted_mean)**2 + group['std']**2)).sum()) / group['count'].sum()
    return weighted_variance**0.5

# Compute the weighted mean first
weighted_mean = (nodes_desc['count' ] * nodes_desc['mean']).sum() / nodes_desc['count'].sum()

# Apply the weighted std function to each group
weighted_stds = nodes_desc.groupby('Bin').apply(weighted_std, weighted_mean=weighted_mean).reset_index(name='Weighted Std')

In [ ]:
weighted_stds

In [ ]:
# Remove rows where the mean or standard deviation is NaN (e.g., row with index 15)
df = df.dropna(subset=['Mean', 'Std'])

# Group by the 'Bin' column
def weighted_std(group, weighted_mean):
    # Compute the weighted variance
    weighted_variance = ((group['Count'] * ((group['Mean'] - weighted_mean)**2 + group['Std']**2)).sum()) / group['Count'].sum())
    return weighted_variance**0.5

# Compute the weighted mean first
weighted_mean = (df['Count'] * df['Mean']).sum() / df['Count'].sum()

# Compute the weighted standard deviation first
weighted_stds = df.groupby('Bin').apply(weighted_std, weighted_mean=weighted_mean).reset_index(name='Weighted Std')

# Calculate the weighted standard error for each bin (Std / sqrt(Count))
weighted_stds['Weighted Std Error'] = weighted_stds['Weighted Std'] / np.sqrt(df.groupby('Bin')['Count'].sum().values)

# Print the results
print(weighted_stds)

In [ ]:
# nodes_desc[nodes_desc['Bin'] == '(100, 150]']

In [ ]:
# # Make nodes dataframe
# d = []
# for file in nodes_quant:
#     data = pd.read_csv(os.path.join(data_path, file))
#     d.append(data)
# nodes_quant = pd.concat(d, axis=0, ignore_index=True)#.rename(columns={'Unnamed: 0': 'stat'})

In [ ]:
# nodes_quant = nodes_quant[nodes_quant['with_zero'] == 1]

In [ ]:
# nodes_quant_binned = nodes_quant.groupby(['Bin', 'quantile']).mean().reset_index().drop(columns=['Unnamed: 0', 'with_zero'])

In [ ]:
# nodes_quant_binned

In [ ]:
# nodes_quant_binned[nodes_quant_binned['Bin'] == '(0, 10]']#.coverage.to_list()

In [ ]:
# Example: two groups of coverage data for two rivers
# Coverage values for River 1 and River 2 (from the data you provided)
river1_coverage = [0.0, 0.000356, 0.00166, 0.0042, 0.00691, 0.00935, 0.01078, 0.01217, 0.01308, 0.01471]  # And more...
river2_coverage = [0.0, 0.00205, 0.00407, 0.00496, 0.00627, 0.00766, 0.01038, 0.01121, 0.0121, 0.01261]  # And more...

# Combine these two rivers' data into a list of groups
data = [river1_coverage, river2_coverage]

In [ ]:
# Example: two groups of coverage data for two rivers
# Coverage values for River 1 and River 2 (from the data you provided)
river1_coverage = [0.0, 0.000356, 0.00166, 0.0042, 0.00691, 0.00935, 0.01078, 0.01217, 0.01308, 0.01471]  # And more...
river2_coverage = [0.0, 0.00205, 0.00407, 0.00496, 0.00627, 0.00766, 0.01038, 0.01121, 0.0121, 0.01261]  # And more...

# Combine these two rivers' data into a list of groups
data = [river1_coverage, river2_coverage]

# Create a boxplot
plt.boxplot(data, tick_labels=['River 1', 'River 2'])

# Set titles and labels
plt.title('Coverage Comparison between Rivers')
plt.ylabel('Coverage Value')

# Show plot
plt.show()

In [ ]:
# node_w_zero = node[node['with_zero'] == 1]

In [ ]:
# node_w_zero

In [ ]:
# node_mean = node[node['stat'] == 'mean'].drop(columns='stat').mean()

In [ ]:
# node_mean = pd.DataFrame(node_mean, columns=['Pseudo-nodes'])